# Google OR-Tools


In [2]:
"""Simple solve."""
from ortools.sat.python import cp_model


def SimpleSatProgram():
    """Minimal CP-SAT example to showcase calling the solver."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, "x")
    y = model.NewIntVar(0, num_vals - 1, "y")
    z = model.NewIntVar(0, num_vals - 1, "z")

    # Creates the constraints.
    model.Add(x != y)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"x = {solver.Value(x)}")
        print(f"y = {solver.Value(y)}")
        print(f"z = {solver.Value(z)}")
    else:
        print("No solution found.")


SimpleSatProgram()

x = 1
y = 0
z = 0


In [33]:
from ortools.sat.python import cp_model


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f"{v}={self.Value(v)}", end=" ")
        print()

    def solution_count(self):
        return self.__solution_count


def SearchForAllSolutionsSampleSat():
    """Showcases calling the solver to search for all solutions."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    num_vals = 3
    x = model.NewIntVar(0, num_vals - 1, "x")
    y = model.NewIntVar(0, num_vals - 1, "y")
    z = model.NewIntVar(0, num_vals - 1, "z")

    # Create the constraints.
    model.Add(x != y)

    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter([x, y, z])
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True
    # Solve.
    status = solver.Solve(model, solution_printer)

    print(f"Status = {solver.StatusName(status)}")
    print(f"Number of solutions found: {solution_printer.solution_count()}")


SearchForAllSolutionsSampleSat()

x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=0 z=1 
x=1 y=0 z=1 
x=2 y=1 z=1 
x=2 y=1 z=0 
x=2 y=1 z=2 
x=2 y=0 z=2 
x=1 y=0 z=2 
x=0 y=1 z=2 
x=0 y=1 z=1 
x=0 y=2 z=1 
x=0 y=2 z=2 
x=1 y=2 z=2 
x=1 y=2 z=1 
x=1 y=2 z=0 
x=0 y=2 z=0 
x=0 y=1 z=0 
Status = OPTIMAL
Number of solutions found: 18


In [34]:
"""Simple solve."""
from ortools.sat.python import cp_model


def main():
    """Minimal CP-SAT example to showcase calling the solver."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    var_upper_bound = max(50, 45, 37)
    x = model.NewIntVar(0, var_upper_bound, "x")
    y = model.NewIntVar(0, var_upper_bound, "y")
    z = model.NewIntVar(0, var_upper_bound, "z")

    # Creates the constraints.
    model.Add(2 * x + 7 * y + 3 * z <= 50)
    model.Add(3 * x - 5 * y + 7 * z <= 45)
    model.Add(5 * x + 2 * y - 6 * z <= 37)

    model.Maximize(2 * x + 2 * y + 3 * z)

    # Creates a solver and solves the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"Maximum of objective function: {solver.ObjectiveValue()}\n")
        print(f"x = {solver.Value(x)}")
        print(f"y = {solver.Value(y)}")
        print(f"z = {solver.Value(z)}")
    else:
        print("No solution found.")

    # Statistics.
    print("\nStatistics")
    print(f"  status   : {solver.StatusName(status)}")
    print(f"  conflicts: {solver.NumConflicts()}")
    print(f"  branches : {solver.NumBranches()}")
    print(f"  wall time: {solver.WallTime()} s")


if __name__ == "__main__":
    main()

Maximum of objective function: 35.0

x = 7
y = 3
z = 5

Statistics
  status   : OPTIMAL
  conflicts: 0
  branches : 0
  wall time: 0.004396000000000001 s


In [ ]:
# Load SPINs
LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
WITH row WHERE row.FirstName IS NOT NULL
MERGE (s:Student {firstName:row.FirstName, lastName:row.LastName})

LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
WITH row WHERE row.IntensiveChoice03 IS NOT NULL
MERGE (s:SPIN_I {SPIN:row.IntensiveChoice03})

